In [1]:
%pdb

Automatic pdb calling has been turned ON


In [2]:
from __future__ import print_function
from __future__ import division
import torch
import sys
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from torch import autograd
from torch.autograd import Variable
import scipy.misc
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid
import pickle as pkl
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
import tqdm
from tqdm import tqdm
import pickle

PyTorch Version:  1.5.1
Torchvision Version:  0.6.0a0+35d732a


In [3]:
torch.cuda.is_available()

False

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
data_file = "../Data/data/train_data/train_task_1_2_sample.csv"
log_file = "../logs/task_2_log.txt"
batch_size = 32
lr = 0.0001
num_epochs = 100
K = 100
total_q = 28000
total_s = 119000

In [8]:
f = open(log_file,'a')

In [24]:
class Question_Ans(Dataset):
    def __init__(self, filename, mode='train'):
        self.df = pd.read_csv(filename)
        self.questionid = self.df['QuestionId'].values
        self.userid = self.df['UserId'].values
        self.ans = self.df['IsCorrect'].values
        
        self.ans = 2*self.ans - 1
        
        self.length=len(self.ans)
        
        
        if(mode=='train'):
            start=int(0*self.length)
            end=int(0.8*self.length)
        elif(mode=='val'):
            start=int(0.8*self.length)
            end=int(1*self.length)
        else:
            start = 0
            end = int(self.length)
            
            
        self.questionid = self.questionid[start:end]
        self.userid = self.userid[start:end]
        self.ans = self.ans[start:end]
        
        self.length=len(self.ans)
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        
        qid = self.questionid[idx]
        uid = self.userid[idx]
        ans = self.ans[idx]
        return qid,uid,ans

In [25]:
train_dataset = Question_Ans(filename=data_file,mode='train')
val_dataset = Question_Ans(filename=data_file,mode='val')

In [26]:
train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset,batch_size=batch_size, shuffle=False)

In [27]:
dataloader = {}
dataloader['train'] = train_dataloader
dataloader['val'] = val_dataloader

In [9]:
dtype=torch.FloatTensor

In [11]:
df = pd.read_csv(data_file)

In [12]:
df.head()

,index,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue
0,8,12574,11665,18336567,1,2,2
1,12,9952,5685,19507712,1,3,3
2,20,11039,1257,16138813,1,1,1
3,23,9342,7980,4414715,0,2,3
4,30,1332,22156,10626184,0,2,1


In [13]:
df['UserId'].value_counts().describe()

count    19343.000000
mean        75.542057
std         83.315788
min          1.000000
25%         25.000000
50%         47.000000
75%         92.000000
max       1036.000000
Name: UserId, dtype: float64

In [18]:
df['UserId'].value_counts().quantile(0.99)

419.0

In [19]:
data_file = "../Data/data/metadata/answer_metadata_task_1_2.csv"

In [20]:
meta_df = pd.read_csv(data_file)

In [21]:
answer_ids = np.array(df['AnswerId'])

In [23]:
meta_df = meta_df[meta_df['AnswerId'].isin(answer_ids)]

In [26]:
meta_df.head()

,AnswerId,DateAnswered,Confidence,GroupId,QuizId,SchemeOfWorkId
66,19548297.0,2020-03-25 10:43:00.000,75.0,7696,8504,NaN
87,12998343.0,2019-02-19 02:06:00.000,NaN,2580,5006,NaN
88,18430821.0,2019-03-19 07:36:00.000,0.0,5374,5621,8420.0
89,18325173.0,2020-01-07 20:26:00.000,NaN,3382,4922,8420.0
94,12931878.0,2018-12-11 10:14:00.000,NaN,10545,8770,8378.0


In [27]:
meta_df = meta_df[['AnswerId','DateAnswered']]

In [31]:
meta_df['AnswerId'] = meta_df['AnswerId'].astype(int)

In [32]:
meta_df.head()

,AnswerId,DateAnswered
66,19548297,2020-03-25 10:43:00.000
87,12998343,2019-02-19 02:06:00.000
88,18430821,2019-03-19 07:36:00.000
89,18325173,2020-01-07 20:26:00.000
94,12931878,2018-12-11 10:14:00.000


In [39]:
df2 = df.set_index('AnswerId').join(meta_df.set_index('AnswerId'))

In [42]:
df2.head()

,index,QuestionId,UserId,IsCorrect,CorrectAnswer,AnswerValue,DateAnswered
AnswerId,,,,,,,
18336567,8,12574,11665,1,2,2,2020-01-29 21:29:00.000
19507712,12,9952,5685,1,3,3,2019-01-11 09:07:00.000
16138813,20,11039,1257,1,1,1,2019-06-12 10:25:00.000
4414715,23,9342,7980,0,2,3,2020-01-19 22:14:00.000
10626184,30,1332,22156,0,2,1,2018-12-16 22:07:00.000


In [43]:
user_ids = df2['UserId'].unique()

In [47]:
user_mapping = {}
for i,user_id in enumerate(user_ids):
    user_mapping[i] = user_id

In [10]:
# Declare two matrices 
# Q = shape (total_q,K)
# U = shape (total_s,K)
Q = torch.nn.Embedding(total_q,K)
U = torch.nn.Embedding(total_s,K)

Q = Q.to(device)
U = U.to(device)

In [30]:
# embedding = torch.nn.Embedding(5,8)

In [31]:
# embedding(torch.LongTensor(np.array([3,4,3])))

In [33]:
def get_qvector(questions):
    ans = Q(torch.LongTensor(questions).to(device))
    
    ans = ans.to(device)
    return ans

In [34]:
def get_uvector(users):
    ans = U(torch.LongTensor(users).to(device))
    ans = ans.to(device)
    return ans

In [35]:
test1 = torch.randn(32,100)
test2 = torch.randn(32,100)
test1 = torch.unsqueeze(test1,1)
test2 = torch.unsqueeze(test2,2)
print(test1.shape)
print(test2.shape)
res = torch.bmm(test1,test2)
print(res.shape)
res = torch.squeeze(res)
print(res.shape)

torch.Size([32, 1, 100])
torch.Size([32, 100, 1])
torch.Size([32, 1, 1])
torch.Size([32])


In [36]:
def get_score(qvectors,uvectors):
    q_unsq = torch.unsqueeze(qvectors, 1)
    u_unsq = torch.unsqueeze(uvectors, 2)
    score = torch.bmm(q_unsq,u_unsq)
    score = torch.squeeze(score)
    return score

In [37]:
params_to_update = list(Q.parameters()) + list(U.parameters())

In [38]:
criterion = nn.MSELoss()
optimizer = optim.SGD(params=params_to_update,lr=lr, momentum=0.9)

In [39]:
best_loss = 1e9
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        
            
        running_loss = 0.0
        
        # Iterate over data.
        cnt = 0
#         print("*********** entered ",phase,"*********************")
        for questions,users,ans in tqdm(dataloader[phase]):
            
#             print("************ loaded data of one batch*******")
            
            cnt = cnt + 1
            ans = torch.tensor(ans)
            ans = ans.type(dtype)
            ans = ans.to(device)
            
            qvectors = get_qvector(questions)
            uvectors = get_uvector(users)
            
            
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward
            
            scores = get_score(qvectors,uvectors)
            
            loss = criterion(scores,ans)
            
            if phase == 'train':
                loss.backward()
                optimizer.step()
        
            # statistics
            running_loss += loss.item()
            
            # compute correct
            # running_corrects += torch.sum(preds == labels.data)
            
            if (cnt % 1000 == 0 ) :
                print('Batch {} {} Loss: {:.4f} '.format(str(cnt),phase, running_loss/(cnt*batch_size)))
                print('Batch {} {} Loss: {:.4f} '.format(str(cnt),phase, running_loss/(cnt*batch_size)),file=f)
            

        epoch_loss = running_loss / len(dataloader[phase].dataset)

        print(' {} Loss: {:.4f} '.format(phase, epoch_loss))
        print(' {} Loss: {:.4f} '.format(phase, epoch_loss),file=f)
        
        # deep copy the model
        if phase == 'val' and epoch_loss < best_loss:
            best_loss = epoch_loss
            
            q_file = '../Weights/simple_fact_q' + '_best' + '.pkl'
            u_file = '../Weights/simple_fact_u' + '_best' + '.pkl'
            q_f = open(q_file, 'wb')
            u_f = open(u_file, 'wb')
            pickle.dump(Q,q_f)
            pickle.dump(U,u_f)
        
        q_file = '../Weights/simple_fact_q_' + str(epoch+1) + '.pkl'
        u_file = '../Weights/simple_fact_u_' + str(epoch+1) + '.pkl'
        q_f = open(q_file, 'wb')
        u_f = open(u_file, 'wb')
        pickle.dump(Q,q_f)
        pickle.dump(U,u_f)
    print()
time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
print('Best val Loss: {:4f}'.format(best_loss))

print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60),file=f)
print('Best val Loss: {:4f}'.format(best_loss),file=f)

  0%|          | 1/36531 [00:00<1:41:52,  5.98it/s]

Epoch 0/99
----------


  3%|▎         | 1002/36531 [00:50<28:58, 20.44it/s]

Batch 1000 train Loss: 3.1478 


  5%|▌         | 2002/36531 [01:37<34:41, 16.59it/s]

...

100%|██████████| 36531/36531 [34:06<00:00, 17.85it/s]


 train Loss: 3.0913 


NameError: name 'pickle' is not defined

> <ipython-input-39-9f94c2642d04>(70)<module>()
     68         q_f = open(q_file, 'wb')
     69         u_f = open(u_file, 'wb')
---> 70         pickle.dump(Q,q_f)
     71         pickle.dump(U,u_f)
     72     print()



ipdb>  q
